In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from hplib import hplib as hpl
import plotly.graph_objects as go
from PLZtoWeatherRegion import getregion

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('results_summary.csv')
heatpumps=hpl.load_database()
heatpumps=heatpumps[['Manufacturer', 'Model', 'Date', 'SPL indoor [dBA]', 'SPL outdoor [dBA]', 'PSB [W]', 'P_th_h_ref [W]','MAPE_P_el', 'MAPE_COP', 'MAPE_P_th',
       'P_th_c_ref [W]', 'MAPE_P_el_cooling', 'MAPE_EER', 'MAPE_Pdc']]

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Input(
                placeholder='Wohnort',
                type='text',
                value='',
                id='Standort',            
            ),
            html.Div("Gebäudetyp: "),
            dcc.Dropdown(
                df['Gebäudetyp'].unique(),
                'Neubau (150 Qm und 0.6 W/(K*Qm))',
                id='sort2',
            ),
            "PV-Ausrichtung: ",
            dcc.Dropdown(
                df['PV-Ausrichtung'].unique(),
                'Süd',
                id='sort3',
            ),
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            "Strombezugskosten in ct/kWh: ",
            dcc.Slider(28, 40, 1,
               value=31,
               id='strombezugskosten'
            ),
            "Einspeisevergütung in ct/kWh: ",
            dcc.Slider(4, 10, 1,
               value=6,
               id='einspeisevergütung'
            ),
            "Gaskosten in ct/kWh: ",
            dcc.Slider(8, 16, 1,
               value=12,
               id='gas-cost'
            ),
            
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic'}]},
            clickData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic'}]}
        )
    ], style={'width': '100%', 'display': 'inline-block', 'padding': '10 50'}),
    html.Div([
        dcc.Graph(
            id='graph2',
            hoverData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic'}]},
            clickData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic'}]}
        )
    ]),
    html.Div([
        dcc.Markdown(
         id='wp-infos')
    ]),
])

@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('Standort', 'value'),
    Input('sort2', 'value'),
    Input('sort3', 'value'),
    Input('strombezugskosten', 'value'),
    Input('einspeisevergütung', 'value'),
    Input('gas-cost', 'value'),
    )
def update_graph(standort, gebäudetyp,pv,strombezugskosten, einspeisevergütung,gascost):
    dff = df[(df['Standort'] == getregion(standort))&(df['Gebäudetyp']==gebäudetyp)&(df['Jahr']==2015)&(df['Typ']=='durchschnittliches Jahr')&(df['Batteriespeicher [kWh]']==0)&(df['PV-Ausrichtung']==pv)]
    dff['bilanzielle Energiekosten'] = dff['Netzbezug [kWh]'].values * strombezugskosten/100 - dff['Netzeinspeisung [kWh]'].values * einspeisevergütung/100
    dff['Gasheizung ohne PV']=(dff['Endenergie (Heizung) [kWh]']+dff['Endenergie (TWW) [kWh]'])*0.009*gascost+(dff['Endenergie (Strom) [kWh]']-dff['WP-Stromverbrauch [kWh]'])*strombezugskosten/100

    fig = px.scatter(x=dff['WP-Typ'],                    
                    y=dff['bilanzielle Energiekosten'],
                    hover_name=dff['WP-Name'],
                    facet_col=dff['WP-Kategorie'],
            )

    fig.update_xaxes(title='Regelung einstufig/geregelt')

    fig.update_yaxes(title='Kosten für ein Jahr [€]')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 20, 'r': 0},hovermode='closest')
    fig.add_trace(go.Scatter(x=dff['WP-Typ'],y=dff['Gasheizung ohne PV'],hovertext='Gasheizung ohne PV',showlegend=False,))
    return fig

@app.callback(
    Output('graph2', 'figure'),
    Input('crossfilter-indicator-scatter', 'clickData'),
    Input('Standort', 'value'),
    Input('sort2', 'value'),
    Input('sort3', 'value'),
    Input('strombezugskosten', 'value'),
    Input('einspeisevergütung', 'value'),
    )
def update_graph2(wp_name,standort, gebäudetyp, pv, strombezugskosten, einspeisevergütung):
    wpname=wp_name['points'][0]['hovertext']
    dff = df[(df['Standort'] == getregion(standort))&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']==wpname)]
    dff['Kosten [1/Jahr]'] = dff['Netzbezug [kWh]'].values * strombezugskosten/100 - dff['Netzeinspeisung [kWh]'].values * einspeisevergütung/100

    if wpname=='Generic':
        if wp_name['points'][0]['curveNumber']==0:
            if wp_name['points'][0]['x']=='geregelt':
                dff=dff.loc[(dff['WP-Kategorie']=='Luft/Wasser')&(dff['WP-Typ']=='geregelt')]
            else:
                dff=dff.loc[(dff['WP-Kategorie']=='Luft/Wasser')&(dff['WP-Typ']!='geregelt')]
        else:
            if wp_name['points'][0]['x']=='geregelt':
                dff=dff.loc[(dff['WP-Kategorie']!='Luft/Wasser')&(dff['WP-Typ']=='geregelt')]
            else:
                dff=dff.loc[(dff['WP-Kategorie']!='Luft/Wasser')&(dff['WP-Typ']!='geregelt')]
    fig = go.Figure()
    for year in [2015, 2045]:
        for typ in dff['Typ'].unique():
            fig.add_trace(go.Scatter(x = dff.loc[(dff['Jahr']==year)&(dff['Typ']==typ)]['Batteriespeicher [kWh]'], y=dff.loc[(dff['Jahr']==year)&(dff['Typ']==typ)]['Kosten [1/Jahr]'], name=str(year)+' ' + typ))
    # Edit the layout
    fig.update_layout(title='Laufende Kosten für ein Jahr Wärme und Strom mit '+wpname+' Wärmepumpe',
                    xaxis_title='Batteriespeicher [kWh]',
                    yaxis_title='Betriebskosten [€/Jahr]')
    fig.update_layout(yaxis=dict(range=[dff.sort_values('Kosten [1/Jahr]')['Kosten [1/Jahr]'].head(1),dff.sort_values('Kosten [1/Jahr]')['Kosten [1/Jahr]'].tail(1)]))
    return fig

@app.callback(
    Output('wp-infos', 'children'),
    Input('crossfilter-indicator-scatter', 'clickData'),
    Input('graph2', 'clickData'))
def update_table(wp_name, Wp_name):
    wpname=wp_name['points'][0]['hovertext']
    hp=heatpumps.loc[heatpumps['Model']==wpname]
    try:
        samehp=''
        for i in hpl.same_built_type(wpname):
            samehp=samehp+i+' \n'
        samehp=samehp[:-2]
        samehp
        hp['Other names']=samehp
        hp=hp[['Manufacturer', 'Model','Other names', 'SPL indoor [dBA]', 'PSB [W]',
            'MAPE_COP', 'P_th_c_ref [W]']].rename(columns={'SPL indoor [dBA]':'Geräuschpegel in dBA','PSB [W]': 'Standby Power in W', 'MAPE_COP': 'Durchschnittlicher qualitativer Fehler in %','P_th_c_ref [W]': 'Kühlen'})
        if hp.loc[hp.index[0],'Kühlen']>0:
            hp['Kühlen']='Ja'
        else:
            hp['Kühlen']='Nein'
    except:
        pass
    return hp.to_markdown()

if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
strombezugskosten = 31
einspeisevergütung = 6
batcost = 100
pvcost = 12
standort = 4
gebäudetyp = 'Neubau (150 Qm und 0.6 W/(K*Qm))'
pv = 'Süd'
wpname='HA 3-6 O 230V'

In [ ]:
dff.sort_values('Kosten [1/Jahr]')['Kosten [1/Jahr]'].tail(1)

In [ ]:
df['Kosten [1/Jahr]'] = df['Netzbezug [kWh]'] * strombezugskosten/100 - df['Netzeinspeisung [kWh]'] * einspeisevergütung/100 + df['Batteriespeicher [kWh]'] * batcost + df['PV-Ertrag [kWh]']*pvcost/100
dff = df[(df['Standort'] == standort)&(df['Gebäudetyp']==gebäudetyp)&(df['Jahr']==2015)&(df['Typ']=='durchschnittliches Jahr')&(df['Batteriespeicher [kWh]']==0)&(df['PV-Ausrichtung']==pv)]

fig = px.scatter(x=dff['WP-Typ'],                    
                y=dff['Kosten [1/Jahr]'],
                hover_name=dff['WP-Name'],
                facet_col=dff['WP-Kategorie'],
        )

fig.update_xaxes(title='Regelung einstufig/geregelt')

fig.update_yaxes(title='Kosten für ein Jahr [€]')

fig.update_layout(margin={'l': 40, 'b': 40, 't': 20, 'r': 0},hovermode='closest')
fig.add_trace(go.Scatter(x=dff['WP-Typ'],y=dff['Gasheizung ohne PV'],hovertext='Gasheizung ohne PV',showlegend=False))

In [ ]:
df['Kosten [1/Jahr]'] = df['Netzbezug [kWh]'] * strombezugskosten/100 - df['Netzeinspeisung [kWh]'] * einspeisevergütung/100 + df['Batteriespeicher [kWh]'] * batcost + df['PV-Ertrag [kWh]']*pvcost/100
dff = df[(df['Standort'] == standort)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']==wpname)]
fig = go.Figure()
for year in [2015, 2045]:
    for typ in dff['Typ'].unique():
        fig.add_trace(go.Scatter(x = dff.loc[(dff['Jahr']==year)&(dff['Typ']==typ)]['Batteriespeicher [kWh]'], y=dff.loc[(dff['Jahr']==year)&(dff['Typ']==typ)]['Kosten [1/Jahr]'], name=str(year)+' ' + typ))

# Edit the layout
fig.update_layout(title='Kosten des Energiesystems für Wärme und Strom',
                   xaxis_title='Batteriespeicher [kWh]',
                   yaxis_title='Betriebskosten [€/Jahr]')

fig

In [ ]:
dff['Typ'].unique()